# **Pseudo-Labeling**

## _**Import Library**_

In [3]:
# Library umum
import pandas as pd
import torch
from tqdm import tqdm

# Transformers (Hugging Face)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax


## **_load data_**

In [4]:
# Load data hasil preprocessing
df = pd.read_csv("../data/processed/half_cleaned.csv")

# Cek 5 data awal
df.head()

,full_text,normalized
0,@IqbalRa17664055 @rwp1byte aku nyusul kerja aj...,aku menyusul kerja saja mau kaburajadulu bawa ...
1,#KaburAjaDulu bole juga ni hashtag,kaburajadulu boleh juga nih hashtag
2,#KaburAjaDulu,kaburajadulu
3,Ehhh ini Mas Adith #KaburAjaDulu ya? Mohon ma...,eh ini mas adith kaburajadulu ya mohon maaf ya...
4,Dengan ini saya resmikan pemilik hashtag #Kabu...,dengan ini saya resmikan pemilik hashtag kabur...


## _**load model**_

In [5]:
# Gunakan model RoBERTa dari w11wo
model_name = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval();  # masuk ke mode evaluasi

In [6]:
# Lihat mapping label
print(model.config.id2label)


{0: 'Positive', 1: 'Neutral', 2: 'Negative'}


In [7]:
from torch.nn.functional import softmax

def predict_sentiment_custom(text):
    clean_text = text.lower().strip()

    # Jika hanya "kaburajadulu" atau "#kaburajadulu", langsung return label netral (1)
    if clean_text in ["kaburajadulu", "#kaburajadulu"]:
        return 1  # netral

    # Prediksi dengan model seperti biasa
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        predicted = torch.argmax(probs, dim=1).item()
    return predicted


In [8]:
df['normalized'].apply(lambda x: type(x)).value_counts()

normalized
<class 'str'>    17664
Name: count, dtype: int64

In [9]:
# Tampilkan progress bar saat proses banyak data
tqdm.pandas()

# Prediksi label numerik
df['label_num'] = df['normalized'].progress_apply(predict_sentiment_custom)

# Konversi label angka ke teks
label_map = {0: 'positif', 1: 'neutral', 2: 'negatif'}
df['pseudo_label'] = df['label_num'].map(label_map)

# Lihat hasil awal
df[['normalized', 'label_num', 'pseudo_label']].head(10)


100%|██████████| 17664/17664 [14:45<00:00, 19.95it/s]


,normalized,label_num,pseudo_label
0,aku menyusul kerja saja mau kaburajadulu bawa ...,2,negatif
1,kaburajadulu boleh juga nih hashtag,1,neutral
2,kaburajadulu,1,neutral
3,eh ini mas adith kaburajadulu ya mohon maaf ya...,1,neutral
4,dengan ini saya resmikan pemilik hashtag kabur...,1,neutral
5,sampai akhirnya aku bisa diundang lunch party ...,1,neutral
6,bisa kali ya tahun depan ngikut tren kaburajadulu,2,negatif
7,kalau yang berkeluarga dan berhasil kaburajadu...,2,negatif
8,pengin mengajak danilla kaburajadulu ke jepang,1,neutral
9,teman teman software engineer yang pengin kabu...,1,neutral


In [10]:
# Simpan ke file untuk digunakan di model training
df.to_csv("../data/processed/half_cleaned_labeled.csv", index=False)
print("✅ Dataset berhasil disimpan ke half_cleaned_labeled.csv")

✅ Dataset berhasil disimpan ke half_cleaned_labeled.csv


In [11]:
# Mengecek jumlah label 0, 1, dan 2 half cleaned 1.5G
label_counts = df['label_num'].value_counts()

# Menampilkan hasil
print(label_counts)


label_num
2    11759
1     4321
0     1584
Name: count, dtype: int64
